# Installation

In [1]:
! pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\annabelle\appdata\local\temp\pip-req-build-ulp3_dor
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit a6b6f3faaa26f541d9469651451340096b5abc92


  Running command git clone -q https://github.com/JustAnotherArchivist/snscrape.git 'C:\Users\annabelle\AppData\Local\Temp\pip-req-build-ulp3_dor'


In [2]:
import os
import pandas as pd

# Single Scraping Example

### Argument of snscraper using `text-query-tweets`: 
```
'telegram-channel', 'vkontakte-user', 'weibo-user', 'facebook-group', 'instagram-user', 'instagram-hashtag', 'instagram-location', 'reddit-user', 'reddit-subreddit', 'reddit-search', 'twitter-search', 'twitter-tweet', 'facebook-user', 'facebook-community', 'twitter-user', 'twitter-hashtag', 'twitter-list-posts', 'twitter-profile' 
```

For example, we can use "twitter-hashtag" to scrape tweets based the hashtags. 
The command below scrape tweets with hashtags "yoga" or "exercise" between the date "2021-03-01" and "2021-05-20" capped at maximum 2000 tweets.

In [3]:
arg = "twitter-hashtag"
os.system(f'snscrape --jsonl --progress --max-results 2000 --since 2020-03-01 {arg} "yoga exercise until:2021-05-20" > text-query-tweets.json')

0

### Convert JSON file to CSV

In [4]:
# Read the json generated from the cell above and create a pandas dataframe
tweets_df = pd.read_json('text-query-tweets.json', lines=True)

In [5]:
tweets_df.columns

Index(['_type', 'url', 'date', 'content', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags'],
      dtype='object')

In [6]:
df = tweets_df[["id", "url", "date", "content",
                "hashtags", "cashtags", "media", "lang"]]

In [7]:
df.to_csv('tweet_data_single.csv')

# Bulk Scraping Example

The code below run the tweets scraping for different hashtag and time intervals and save them in a Dataframe, at last to a CSV file. 

For example, we provide these hashtags, "valentines", "christmas", "halloween". The command below scrape tweets with the hashtags provided between the date "2021-08-01" and "2021-11-01" capped at maximum 100 tweets each month for each hashtag.

In [8]:
hashtag = ["valentines", "christmas", "halloween"] #hashtag to scrape
date_interval = ["2021-08-01", "2021-09-01", "2021-10-01", "2021-11-01"] #to ensure there are tweets from different month

In [9]:
data = None
for i in range(len(date_interval)-1):
    for hash in hashtag:
        os.system(
            f'snscrape --jsonl --progress --max-results 100 --since {date_interval[i]} {arg} "{hash} until:{date_interval[i+1]}" > text-query-tweets.json')
        tweets_df = pd.read_json('text-query-tweets.json', lines=True)
        df = tweets_df[["id", "url", "date", "content",
                        "hashtags", "cashtags", "media", "lang"]]
        if data is None:
            data = df
        else:
            data = data.append(df)
        print(f"{hash} ({date_interval[i]}->{date_interval[i+1]}): {len(df)} ")

valentines (2021-08-01->2021-09-01): 100 
christmas (2021-08-01->2021-09-01): 100 
halloween (2021-08-01->2021-09-01): 100 
valentines (2021-09-01->2021-10-01): 100 
christmas (2021-09-01->2021-10-01): 100 
halloween (2021-09-01->2021-10-01): 100 
valentines (2021-10-01->2021-11-01): 100 
christmas (2021-10-01->2021-11-01): 100 
halloween (2021-10-01->2021-11-01): 100 


In [10]:
len(data)

900

In [11]:
data.to_csv('tweet_data.csv')